<a href="https://colab.research.google.com/github/maverick98/CDS/blob/main/Mano_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gensim
!pip install python-levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.0/174.0 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.5 MB/s eta 0:00:00


In [3]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

--2023-02-03 09:36:50--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45409631 (43M) [application/x-gzip]
Saving to: ‘reviews_Cell_Phones_and_Accessories_5.json.gz’

reviews_Cell_Phones 100%[===================>]  43.31M  11.2MB/s    in 4.1s    

2023-02-03 09:36:54 (10.6 MB/s) - ‘reviews_Cell_Phones_and_Accessories_5.json.gz’ saved [45409631/45409631]



In [6]:
!gzip -d /content/reviews_Cell_Phones_and_Accessories_5.json.gz

In [9]:
import gensim
import numpy as np
import pandas as pd

In [10]:
df=pd.read_json('/content/reviews_Cell_Phones_and_Accessories_5.json',lines=True)

In [11]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [12]:
df.shape

(194439, 9)

In [15]:
df.reviewText[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [17]:
gensim.utils.simple_preprocess("They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again")

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [18]:
reviewText=df['reviewText'].apply(gensim.utils.simple_preprocess)
reviewText

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [20]:
model=gensim.models.Word2Vec(  window = 10, min_count=2,workers=4)

In [22]:
model.build_vocab(reviewText,progress_per=1000)

In [23]:
model.epochs

5

In [25]:
model.corpus_count

194439

In [26]:
model.train(reviewText,total_examples=model.corpus_count,epochs=model.epochs)

(61509536, 83868975)

In [27]:
model.save('/content/gensim.model')

In [33]:
model.wv.most_similar('cute')

[('adorable', 0.8783262968063354),
 ('girly', 0.7869681119918823),
 ('sparkly', 0.7323333621025085),
 ('bling', 0.717789888381958),
 ('sparkles', 0.6920074820518494),
 ('blingy', 0.6570302844047546),
 ('fashionable', 0.6506723761558533),
 ('stylish', 0.6328210234642029),
 ('glitter', 0.6286778450012207),
 ('beautiful', 0.6276790499687195)]

In [34]:
model.wv.similarity(w1="cute",w2="beautiful")

0.627679

In [35]:
model.wv.similarity(w1="cute",w2="equation")

-0.19647515

In [36]:
model.wv.similarity(w1="good",w2="bad")

0.60165733